<a href="https://colab.research.google.com/github/masadream/data_analytics/blob/master/Causal_Inference_studysession/Ch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2.3

##### モジュール準備

In [0]:
!pip install rdata

import pandas as pd
import rdata
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline

##### データセット準備

In [0]:
cd drive/'My Drive'/'Colab Notebooks'/'効果検証入門_輪読会'

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/効果検証入門_輪読会'
/content/drive/My Drive/Colab Notebooks/効果検証入門_輪読会


In [0]:
!git clone https://github.com/itamarcaspi/experimentdatar/

fatal: destination path 'experimentdatar' already exists and is not an empty directory.


##### Rデータセットを読み込む

In [0]:
parsed = rdata.parser.parse_file('./experimentdatar/data/vouchers.rda')
converted = rdata.conversion.convert(parsed)
vouchers = converted['vouchers']
vouchers.head(10)

/usr/local/lib/python3.6/dist-packages/rdata/conversion/_conversion.py:495: UserWarning: Missing constructor for R class "tbl_df". The constructor for class "tbl" will be used instead.
  stacklevel=1)
/usr/local/lib/python3.6/dist-packages/rdata/conversion/_conversion.py:495: UserWarning: Missing constructor for R class "tbl". The constructor for class "data.frame" will be used instead.
  stacklevel=1)


,ID,BOG95SMP,BOG97SMP,JAM93SMP,SEX,AGE,AGE2,HSVISIT,SCYFNSH,INSCHL,PRSCH_C,PRSCHA_1,PRSCHA_2,VOUCH0,BOG95ASD,BOG97ASD,JAM93ASD,DBOGOTA,DJAMUNDI,D1995,D1997,RESPONSE,TEST_TAK,SEX_NAME,SVY,D1993,PHONE,DAREA1,DAREA2,DAREA3,DAREA4,DAREA5,DAREA6,DAREA7,DAREA8,DAREA9,DAREA10,DAREA11,DAREA12,DAREA13,...,DMONTH4,DMONTH5,DMONTH6,DMONTH7,DMONTH8,DMONTH9,DMONTH10,DMONTH11,DMONTH12,BOG95,BOG97,MOM_SCH,MOM_AGE,MOM_MW,DAD_SCH,DAD_AGE,DAD_MW,SEX2,STRATA1,STRATA2,STRATA3,STRATA4,STRATA5,STRATA6,STRATAMS,REPT6,TOTSCYRS,HASCHILD,MARRIED,WORKING,REPT,NREPT,FINISH6,FINISH7,FINISH8,SEX_MISS,USNGSCH,HOURSUM,TAB3SMPL,WORKING3
0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,1.0,0.0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,0.0,0.0,0.0,1.0,NaN,12.0,NaN,5.0,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,0.0,0.0,0.0,0.0,NaN,13.0,NaN,5.0,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,1.0,0.0,0.0,0.0,14.0,12.0,0.0,8.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0,45.0,0.0,2.0,45.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
4,4.0,1.0,0.0,0.0,1.0,14.0,12.0,0.0,8.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.0,36.0,0.0,11.0,43.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
5,5.0,1.0,0.0,0.0,0.0,14.0,12.0,0.0,8.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0,38.0,0.0,5.0,41.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
6,6.0,1.0,0.0,0.0,0.0,12.0,10.0,0.0,7.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0,30.0,0.0,8.0,36.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
7,7.0,0.0,0.0,0.0,1.0,NaN,13.0,NaN,5.0,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8.0,0.0,0.0,0.0,1.0,NaN,12.0,NaN,5.0,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9.0,0.0,0.0,0.0,1.0,NaN,13.0,NaN,5.0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#2.3.2

In [0]:
# 回帰式の準備
# 回帰式で使う文字列の準備
formula_x_base = 'VOUCH0'
formula_x_covariate = ['SVY', 'HSVISIT', 'AGE', 'STRATA1', 'STRATA2', 'STRATA3', 'STRATA4', 'STRATA5', 'STRATA6', 'STRATAMS','D1993', 'D1995', 'D1997', 'DMONTH1', 'DMONTH2', 'DMONTH3', 'DMONTH4', 'DMONTH5', 'DMONTH6','DMONTH7', 'DMONTH8', 'DMONTH9', 'DMONTH10', 'DMONTH11', 'DMONTH12', 'SEX2']
formula_y = ["TOTSCYRS","INSCHL","PRSCH_C","USNGSCH","PRSCHA_1","FINISH6","FINISH7","FINISH8","REPT6","REPT","NREPT","MARRIED","HASCHILD","HOURSUM","WORKING3"]

In [0]:
# 回帰分析をまとめて実行させる関数を定義

# 共変量なし
def exec_pred_base(data, f_y, f_x):
    y = data[f_y]
    x = sm.add_constant(data[f_x])
    result = sm.OLS(y, x).fit()
    summary = result.summary().tables[1].as_html()
    summary = pd.read_html(summary, header=0, index_col=0)[0]
    summary = summary.loc["VOUCH0"]
    summary.name =  str(f_y) + "_base"
    return summary

# 共変量あり
def exec_pred_covariate(data, f_y, f_x_base, f_x_covariate):
    y = data[f_y]
    x = sm.add_constant(data[[f_x_base] + formula_x_covariate])
    result = sm.OLS(y, x).fit()
    summary = result.summary().tables[1].as_html()
    summary = pd.read_html(summary, header=0, index_col=0)[0]
    summary = summary.loc["VOUCH0"]
    summary.name =  str(f_y) + "_covariate"
    return summary

In [0]:
# データソースの準備
regression_data = vouchers[(vouchers["TAB3SMPL"] == 1) & (vouchers["BOG95SMP"] == 1)]

In [0]:
# 回帰分析実行
df_results = []

for f_y in formula_y:
    result_reg = exec_pred_base(regression_data,f_y,formula_x_base)
    df_results.append(result_reg)

for f_y in formula_y:
    result_reg = exec_pred_covariate(regression_data,f_y,formula_x_base,formula_x_covariate)
    df_results.append(result_reg)

/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1561: RuntimeWarning: invalid value encountered in subtract
  self.params, self.model.wexog)
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:425: RuntimeWarning: invalid value encountered in multiply
  avg = np.multiply(a, wgt, dtype=result_dtype).sum(axis)/scl
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1594: RuntimeWarning: invalid value encountered in subtract
  return np.sum(weights * (model.endog - mean)**2)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:861: RuntimeWarning: invalid value encountered in subtract
  resid = self.endog - np.dot(self.exog, params)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid v

In [0]:
# 結果をマージ
regression_results = pd.concat(df_results, axis=1).T

# 2.3.3

In [0]:
# 通学率と奨学金の利用
using_voucher_results = regression_results.loc[regression_results.index.str.contains('PRSCHA_1|USNGSCH', regex=True)]
using_voucher_results.sort_index()

,coef,std err,t,P>|t|,[0.025,0.975]
PRSCHA_1_base,0.0629,0.017,3.731,0.000,0.030,0.096
PRSCHA_1_covariate,0.0574,0.017,3.385,0.001,0.024,0.091
USNGSCH_base,0.5089,0.023,22.107,0.000,0.464,0.554
USNGSCH_covariate,0.5042,0.023,22.007,0.000,0.459,0.549


# 2.3.4

# 2.3.5

In [0]:
# 性別による効果差
#データソースの準備
data_tbl4_bog95 = vouchers[(vouchers["BOG95SMP"] == 1) & (vouchers["TAB3SMPL"] == 1) & (~vouchers["SCYFNSH"].isna()) & (~vouchers["FINISH6"].isna()) & (~vouchers["PRSCHA_1"].isna()) & (~vouchers["REPT6"].isna()) & (~vouchers["NREPT"].isna()) & (~vouchers["INSCHL"].isna()) & (~vouchers["FINISH7"].isna()) & (~vouchers["PRSCH_C"].isna()) & (~vouchers["FINISH8"].isna()) & (~vouchers["PRSCHA_2"].isna()) & (~vouchers["TOTSCYRS"].isna()) & (~vouchers["REPT"].isna())]
data_tbl4_bog95 = data_tbl4_bog95[['VOUCH0', 'SVY', 'HSVISIT', 'DJAMUNDI', 'PHONE', 'AGE', 'STRATA1', 'STRATA2', 'STRATA3', 'STRATA4', 'STRATA5', 'STRATA6', 'STRATAMS', 'DBOGOTA', 'D1993', 'D1995', 'D1997', 'DMONTH1', 'DMONTH2', 'DMONTH3', 'DMONTH4', 'DMONTH5', 'DMONTH6', 'DMONTH7', 'DMONTH8', 'DMONTH9', 'DMONTH10', 'DMONTH11', 'DMONTH12', 'SEX_MISS', 'FINISH6', 'FINISH7', 'FINISH8', 'REPT6', 'REPT', 'NREPT', 'SEX2', 'TOTSCYRS', 'MARRIED', 'HASCHILD', 'HOURSUM','WORKING3', 'INSCHL','PRSCH_C','USNGSCH','PRSCHA_1']]
data_tbl4_bog95

,VOUCH0,SVY,HSVISIT,DJAMUNDI,PHONE,AGE,STRATA1,STRATA2,STRATA3,STRATA4,STRATA5,STRATA6,STRATAMS,DBOGOTA,D1993,D1995,D1997,DMONTH1,DMONTH2,DMONTH3,DMONTH4,DMONTH5,DMONTH6,DMONTH7,DMONTH8,DMONTH9,DMONTH10,DMONTH11,DMONTH12,SEX_MISS,FINISH6,FINISH7,FINISH8,REPT6,REPT,NREPT,SEX2,TOTSCYRS,MARRIED,HASCHILD,HOURSUM,WORKING3,INSCHL,PRSCH_C,USNGSCH,PRSCHA_1
3,1.0,0.0,0.0,0.0,1.0,14.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0,1.0,14.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
5,0.0,0.0,0.0,0.0,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
6,0.0,0.0,0.0,0.0,1.0,12.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
10,1.0,0.0,0.0,0.0,1.0,14.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3997,1.0,0.0,1.0,0.0,1.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4006,1.0,0.0,0.0,0.0,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
4022,0.0,1.0,0.0,0.0,1.0,16.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
4023,0.0,1.0,0.0,0.0,1.0,16.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0


In [0]:
# 女子生徒のみのデータで回帰分析
regression_data = data_tbl4_bog95[data_tbl4_bog95["SEX2"] == 0]

# 回帰分析実行
df_results = []

for f_y in formula_y:
    result_reg = exec_pred_base(regression_data,f_y,formula_x_base)
    df_results.append(result_reg)

for f_y in formula_y:
    result_reg = exec_pred_covariate(regression_data,f_y,formula_x_base,formula_x_covariate)
    df_results.append(result_reg)

# 結果をマージ
df_results_female = pd.concat(df_results, axis=1).T
df_results_female["gender"] = "female"

/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1561: RuntimeWarning: invalid value encountered in subtract
  self.params, self.model.wexog)
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:425: RuntimeWarning: invalid value encountered in multiply
  avg = np.multiply(a, wgt, dtype=result_dtype).sum(axis)/scl
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1594: RuntimeWarning: invalid value encountered in subtract
  return np.sum(weights * (model.endog - mean)**2)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:861: RuntimeWarning: invalid value encountered in subtract
  resid = self.endog - np.dot(self.exog, params)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid v

In [0]:
# 男子生徒のみのデータで回帰分析
regression_data = data_tbl4_bog95[data_tbl4_bog95["SEX2"] == 1]

# 回帰分析実行
df_results = []

for f_y in formula_y:
    result_reg = exec_pred_base(regression_data,f_y,formula_x_base)
    df_results.append(result_reg)

for f_y in formula_y:
    result_reg = exec_pred_covariate(regression_data,f_y,formula_x_base,formula_x_covariate)
    df_results.append(result_reg)

# 結果をマージ
df_results_male = pd.concat(df_results, axis=1).T
df_results_male["gender"] = "male"

In [0]:
# 男女の結果をマージ
regression_results = pd.concat([df_results_female,df_results_male], axis=0)

In [0]:
regression_results.loc[regression_results.index.str.contains('(PRSCHA_1_covariate|USNGSCH_covariate)', regex=True)].sort_index()

/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,coef,std err,t,P>|t|,[0.025,0.975],gender
PRSCHA_1_covariate,0.0229,0.023,1.011,0.312,-0.022,0.067,female
PRSCHA_1_covariate,0.0902,0.025,3.555,0.000,0.040,0.140,male
USNGSCH_covariate,0.5436,0.032,16.992,0.000,0.481,0.606,female
USNGSCH_covariate,0.4677,0.033,14.159,0.000,0.403,0.533,male
